<a href="https://colab.research.google.com/github/dSudario9/Modelos-Supervisionados-Diversos/blob/main/Classifica%C3%A7%C3%A3o_tunning_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*** Banco Thera Banck***
 
 ***O departamento quer Contruir um modelo de classificação  que os ajude a identificar os clientes potenciais com maior probabilidade de adquirir o emprestimo. Isso aumentará a taxa de sucesso e ao mesmo tempo reduzirá o custo da campanha. ***

A base de dados historica contém informações pessoas dos clientes e informações sobre produtos contratados. O arquivo também contém um dicionario das variaveis.

In [ ]:
# Import das bibliotecas basicas. 
import pandas as pd 

In [ ]:
# Carregando base de dados

df = pd.read_excel("/content/Bank_Personal_Loan_Modelling.xlsx", sheet_name='Data')

In [ ]:
df.head() # Visualizar df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
# Através da análise das variáveis e do dicionário, algums informações relevantes:


# Variáveis categóricas:
# Personal Loan (Empréstimo Pessoal) - Este cliente aceitou o empréstimo pessoal oferecido na última campanha? Esta é a nossa variável alvo
# Securities Account (Conta de títulos) - O cliente possui conta de títulos no banco?
# CD Account (Conta de CD) - O cliente tem uma conta de certificado de depósito (CD) no banco?
# Online - O cliente usa serviços de banco pela Internet?
# Credit Card - (Cartão de crédito) - O cliente usa um cartão de crédito emitido pelo UniversalBank?

# Variáveis contínuas:

# Age (Idade) Idade do cliente
# Experience (Experiência) - anos de experiência
# Income (Renda) - Renda anual em dólares
# CCAvg (CCAvg) - gasto médio com cartão de crédito
# Mortage (Hipoteca) - Valor da hipoteca da casa

# Variáveis ​​categóricas ordinais:
# Family (Família) - tamanho da família do cliente
# Education (Educação) - nível de educação do cliente


# A variável ID não adiciona nenhuma informação interessante. 
# Não há associação entre o ID de cliente de uma pessoa e o empréstimo, também não fornece nenhuma conclusão geral para futuros clientes de empréstimo em potencial. 
# Podemos desconsiderar essas informações para o modelo.
# Zip code também pode ser descosiderada, uma vez que a informação de distnância entre zip code pode não ser realidade



In [ ]:
#Excluindo variáveis não úteis

df = df.drop(columns=["ID", "ZIP Code"])

In [ ]:
# Estatística básica das variáveis

df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,5000.0,45.338400,11.463166,23.0,35.0,45.0,55.0,67.0
Experience,5000.0,20.104600,11.467954,-3.0,10.0,20.0,30.0,43.0
Income,5000.0,73.774200,46.033729,8.0,39.0,64.0,98.0,224.0
Family,5000.0,2.396400,1.147663,1.0,1.0,2.0,3.0,4.0
CCAvg,5000.0,1.937913,1.747666,0.0,0.7,1.5,2.5,10.0
Education,5000.0,1.881000,0.839869,1.0,1.0,2.0,3.0,3.0
Mortgage,5000.0,56.498800,101.713802,0.0,0.0,0.0,101.0,635.0
Personal Loan,5000.0,0.096000,0.294621,0.0,0.0,0.0,0.0,1.0
Securities Account,5000.0,0.104400,0.305809,0.0,0.0,0.0,0.0,1.0
CD Account,5000.0,0.060400,0.238250,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Distruibuição da variáveis target 
df["Personal Loan"].value_counts()


0    1500
1     480
Name: Personal Loan, dtype: int64

In [ ]:
# Pelo fato da base ser desbalanceada, não iremos usar toda a classe 0.
# Iremos pegar uma amostra e juntar com o registros da classe 1 para podermos modelar de forma mais justa.

df_classe_1 = df[df["Personal Loan"] == 1]
df_classe_0 = df[df["Personal Loan"] == 0].sample(n=1500)
df = df_classe_1.append(df_classe_0)

In [ ]:
# Dividindo preditores (X) e variável target (Y)

X = df.drop(columns=["Personal Loan"])
Y = df["Personal Loan"]

In [ ]:
# Dividindo a base em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test: {X_test.shape}")
print(f"Shape y_test: {y_test.shape}")

Shape X_train: (1584, 11)
Shape y_train: (1584,)
Shape X_test: (396, 11)
Shape y_test: (396,)


In [ ]:
# Import modelo Selecionado  GridSearchCV
from sklearn.model_selection import GridSearchCV


In [ ]:
# Dicionario para nossos parametros buscar as melhores metricas.
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [None, 80, 110],
    'max_features': [None, 2, 3, 4],
    'min_samples_leaf': [1,2,3],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [50,100, 200],
    'criterion':['gini', 'entropy']
}

In [ ]:
# Classificador RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Fazendo Busca de Imperparametros 
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, 
                          cv=3, n_jobs=-1, verbose=2)

# Selecionando os melhores parametros

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 80, 110],
                         'max_features': [None, 2, 3, 4],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [50, 100, 200]},
             verbose=2)

In [ ]:
# Visualizando os melhores parametros encontrados. 
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 100}

In [ ]:
# Classificador Random Forest
# Configurando com os melhores parametros encontrados pelo grid search

clf = RandomForestClassifier( bootstrap=False,
                              criterion='gini',
                              max_depth=110,
                              max_features=3,
                              min_samples_leaf=1,
                              min_samples_split=4,
                             n_estimators=50) 
# Treinando Classificador 
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, max_depth=110, max_features=3,
                       min_samples_split=4, n_estimators=50)

In [ ]:
# Resultado do Modelo .
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)

print(f"Acurácia: {accuracy_score(y_test, y_pred)*100}%")

Acurácia: 97.97979797979798%


Como podemos vizualizar neste modelo tivemos uma Acurácia: 97.97979797979798%, que faz dele o melhor resultado . 